In [1]:
import numpy as np
from scipy.stats import rayleigh
from scipy.special import digamma,erf
import scipy.io as sio
import pandas as pd

In [2]:
def parseMat(data):
    dataSeqs= IPTVData['Seqs']  
    mdtype = dataSeqs.dtype  
    seqs = {n: dataSeqs[n].flatten() for n in mdtype.names}
    dataStats = IPTVData['Stats']  
    mdtype = dataSeqs.dtype  
    stats = {n: dataSeqs[n].flatten() for n in mdtype.names}
    return seqs,stats

In [3]:
class Model:
    def __init__(self,seqs,K):
            self.N = len(seqs['Time'])
            D= np.zeros(self.N)
            sigma = np.zeros(self.N)
            Tmax = np.zeros(self.N)
            for i in range(self.N):
                D[i] = np.max(seqs['Mark'][i])
                sigma[i] = (4*(np.std(seqs['Time'][i])**5) / (3* len(seqs['Time'][i])))**0.2
                Tmax[i] = seqs['Time'][i][-1][0]
            self.D = np.int(np.max(D))
            Tmax = np.mean(Tmax)
            self.w = np.mean(sigma)
            self.landmark = self.w* range(np.int(np.ceil(Tmax/self.w)))
            self.alpha = np.ones(K)
            M = len(self.landmark)
            self.beta = np.ones((self.D,M,K,self.D)) / (M*(self.D**2))
            self.b = np.ones((self.D,K))/self.D
            label = np.round(K * np.random.rand(self.N))
            self.r = np.zeros((self.N,K))
            for k in range(K):
                self.r[label==k,k] = 1
            self.kernel = "gauss"
class Alg:
    def __init__ (self, outer = 8,rho = 0.1,inner = 5,thres =  1e-5,Tmax = []):
        self.outer = outer
        self.rho = rho
        self.inner = inner
        self.thres = thres
        self.Tmax = Tmax

In [4]:
linkedInData = sio.loadmat('Data/LinkedinData.mat')
IPTVData = sio.loadmat('Data/IPTVData.mat')
IPTVSeqs,IPTVStats = parseMat(IPTVData)
linkedInSeqs,linedInStats = parseMat(linkedInData)

In [15]:
def kernel_int(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance -= landmark
    G = 0
    if model.kernel == 'gauss':
        G = 0.5 *(erf(distance/(np.sqrt(2))*model.w)
                  + erf(landmark/(np.sqrt(2))*model.w))
    elif model.kernel == 'exp':
        G = 1 - np.exp(-model.w * (distance-landmark));
        G[G<0] = 0
    return G

In [50]:
def kernel(dt,model):
    distance = np.tile(dt.flatten(),(len(model.landmark),1))
    landmark = np.tile(model.landmark,(len(dt.flatten()),1)).T
    distance -= landmark
    print("distance: ")
    print(distance.shape)
    g = 0
    if model.kernel == 'gauss':
        g = np.exp(-(distance**2)/(2*(model.w**2))/(np.sqrt(2*np.pi)*model.w))
    elif model.kernel == 'exp':
        g = model.w * np.exp(-model.w * distance)
        g[g>1] = 0
    return g

In [7]:
def E_log_pi(alpha):
    return digamma(alpha)  - digamma(np.sum(alpha))

In [49]:
K = 3
seqs = IPTVSeqs
model = Model(seqs,K)
alg = Alg()
Nk = np.sum(model.r,0)
alpha = model.alpha + Nk
LL = E_log_pi(alpha)
EX = np.zeros((model.N,K))
for c in range(model.N):
    time = seqs['Time'][c]
    event = seqs['Mark'][c]
    Tstart = seqs['Start'][c]
    
    if not alg.Tmax:
        Tstop = seqs['Stop'][c]
    else:
        Tstop = alg.Tmax
        indt = time < alg.Tmax
        time = time[indt]
        event = event[indt]
    N = len(time)
    G = kernel_int(Tstop-time,model)
    for i in range(N):
        ui = event[i]
        ti = time[i]
        
        E_lambda_i = np.sqrt(np.pi/2) * model.b[ui-1,:]
        V_lambda_i = 2 - np.sqrt(np.pi/2) * (model.b[ui-1,:]**2)
        
        if i > 0:
            uj = event[0:i]
            tj = time[0:i]
            
            gij = kernel(ti-tj,model)
            auiuij = model.beta[uj-1,:,:,ui-1]
#             print(auiuij.shape)
#             print(gij)
            
        
    break

distance: 
(17, 1)
distance: 
(17, 2)
distance: 
(17, 3)
distance: 
(17, 4)
distance: 
(17, 5)
distance: 
(17, 6)
distance: 
(17, 7)
distance: 
(17, 8)
distance: 
(17, 9)
distance: 
(17, 10)
distance: 
(17, 11)
distance: 
(17, 12)
distance: 
(17, 13)
distance: 
(17, 14)
distance: 
(17, 15)
distance: 
(17, 16)
distance: 
(17, 17)
distance: 
(17, 18)
distance: 
(17, 19)
distance: 
(17, 20)
distance: 
(17, 21)
distance: 
(17, 22)
distance: 
(17, 23)
distance: 
(17, 24)
distance: 
(17, 25)
distance: 
(17, 26)
distance: 
(17, 27)
distance: 
(17, 28)
distance: 
(17, 29)
distance: 
(17, 30)
distance: 
(17, 31)
distance: 
(17, 32)
distance: 
(17, 33)
distance: 
(17, 34)
distance: 
(17, 35)
distance: 
(17, 36)
distance: 
(17, 37)
distance: 
(17, 38)
distance: 
(17, 39)
distance: 
(17, 40)
distance: 
(17, 41)
distance: 
(17, 42)
distance: 
(17, 43)
distance: 
(17, 44)
distance: 
(17, 45)
distance: 
(17, 46)
distance: 
(17, 47)
distance: 
(17, 48)
distance: 
(17, 49)
distance: 
(17, 50)
distance:

distance: 
(17, 1143)
distance: 
(17, 1144)
distance: 
(17, 1145)
distance: 
(17, 1146)
distance: 
(17, 1147)
distance: 
(17, 1148)
distance: 
(17, 1149)
distance: 
(17, 1150)
distance: 
(17, 1151)
distance: 
(17, 1152)
distance: 
(17, 1153)
distance: 
(17, 1154)
distance: 
(17, 1155)
distance: 
(17, 1156)
distance: 
(17, 1157)
distance: 
(17, 1158)
distance: 
(17, 1159)
distance: 
(17, 1160)
distance: 
(17, 1161)
distance: 
(17, 1162)
distance: 
(17, 1163)
distance: 
(17, 1164)
distance: 
(17, 1165)
distance: 
(17, 1166)
distance: 
(17, 1167)
distance: 
(17, 1168)
distance: 
(17, 1169)
distance: 
(17, 1170)
distance: 
(17, 1171)
distance: 
(17, 1172)
distance: 
(17, 1173)
distance: 
(17, 1174)
distance: 
(17, 1175)
distance: 
(17, 1176)
distance: 
(17, 1177)
distance: 
(17, 1178)
distance: 
(17, 1179)
distance: 
(17, 1180)
distance: 
(17, 1181)
distance: 
(17, 1182)
distance: 
(17, 1183)
distance: 
(17, 1184)
distance: 
(17, 1185)
distance: 
(17, 1186)
distance: 
(17, 1187)
distance: 

distance: 
(17, 2023)
distance: 
(17, 2024)
distance: 
(17, 2025)
distance: 
(17, 2026)
distance: 
(17, 2027)
distance: 
(17, 2028)
distance: 
(17, 2029)
distance: 
(17, 2030)
distance: 
(17, 2031)
distance: 
(17, 2032)
distance: 
(17, 2033)
distance: 
(17, 2034)
distance: 
(17, 2035)
distance: 
(17, 2036)
distance: 
(17, 2037)
distance: 
(17, 2038)
distance: 
(17, 2039)
distance: 
(17, 2040)
distance: 
(17, 2041)
distance: 
(17, 2042)
distance: 
(17, 2043)
distance: 
(17, 2044)
distance: 
(17, 2045)
distance: 
(17, 2046)
distance: 
(17, 2047)
distance: 
(17, 2048)
distance: 
(17, 2049)
distance: 
(17, 2050)
distance: 
(17, 2051)
distance: 
(17, 2052)
distance: 
(17, 2053)
distance: 
(17, 2054)
distance: 
(17, 2055)
distance: 
(17, 2056)
distance: 
(17, 2057)
distance: 
(17, 2058)
distance: 
(17, 2059)
distance: 
(17, 2060)
distance: 
(17, 2061)
distance: 
(17, 2062)
distance: 
(17, 2063)
distance: 
(17, 2064)
distance: 
(17, 2065)
distance: 
(17, 2066)
distance: 
(17, 2067)
distance: 

distance: 
(17, 2704)
distance: 
(17, 2705)
distance: 
(17, 2706)
distance: 
(17, 2707)
distance: 
(17, 2708)
distance: 
(17, 2709)
distance: 
(17, 2710)
distance: 
(17, 2711)
distance: 
(17, 2712)
distance: 
(17, 2713)
distance: 
(17, 2714)
distance: 
(17, 2715)
distance: 
(17, 2716)
distance: 
(17, 2717)
distance: 
(17, 2718)
distance: 
(17, 2719)
distance: 
(17, 2720)
distance: 
(17, 2721)
distance: 
(17, 2722)
distance: 
(17, 2723)
distance: 
(17, 2724)
distance: 
(17, 2725)
distance: 
(17, 2726)
distance: 
(17, 2727)
distance: 
(17, 2728)
distance: 
(17, 2729)
distance: 
(17, 2730)
distance: 
(17, 2731)
distance: 
(17, 2732)
distance: 
(17, 2733)
distance: 
(17, 2734)
distance: 
(17, 2735)
distance: 
(17, 2736)
distance: 
(17, 2737)
distance: 
(17, 2738)
distance: 
(17, 2739)
distance: 
(17, 2740)
distance: 
(17, 2741)
distance: 
(17, 2742)
distance: 
(17, 2743)
distance: 
(17, 2744)
distance: 
(17, 2745)
distance: 
(17, 2746)
distance: 
(17, 2747)
distance: 
(17, 2748)
distance: 

In [9]:
def Expectation_DMHP(Seqs,model,alg):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-9-46420e66f3b4>, line 2)